In [2]:
import daal4py as d4p
import nltk
import string
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import numpy as np
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.preprocessing import Imputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFECV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import scipy.stats as ss
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.manifold import TSNE
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import scipy
import numpy as np, scipy.stats as st
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from scipy import interp
from collections import Counter
# import warnings filter
from warnings import simplefilter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import balanced_accuracy_score

from imblearn.datasets import fetch_datasets
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.metrics import geometric_mean_score
import string
import daal4py as dp
from datetime import date
import daal4py.sklearn as daalsk
import daal4py.sklearn.ensemble as dsk_ensm
import daal4py.sklearn.linear_model as dsk_linear
import daal4py.sklearn.neighbors as daalsknb



Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


In [3]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
class timeit():
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

In [4]:
X_vitals=pd.read_csv('clinical_vitals(1).csv')
y_vitals=pd.read_csv('labels.csv')
X_vitals=X_vitals.drop('Unnamed: 0',axis=1)
X_vitals['label']=y_vitals
X_vitals = X_vitals.drop(['LABEL', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'ETHNICITY', 'DOB','mortality_period'], 1)
X_vitals = pd.get_dummies(X_vitals)
#since SVM built for daal4py doesn't accept labels 0,1, i had to convert them to -1,1
y_vitals = np.where(y_vitals == 0, -1, y_vitals)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_vitals, y_vitals, test_size=0.33, random_state=42,stratify=y_vitals)

In [7]:
X_vitals.columns

Index(['AGE', 'bicarbonate', 'INR', 'MCH', 'AST', 'alkaline phosphatase', 'PH',
       'creatinine', 'platelet', 'PT', 'PTT', 'lymphocytes', 'RBCDW',
       'calcium', 'neutrophils', 'glucose', 'hematocrit', 'hemoglobin',
       'lactate', 'BUN', 'label', 'ADMISSION_TYPE_EMERGENCY',
       'ADMISSION_TYPE_URGENT', 'INSURANCE_Government', 'INSURANCE_Medicaid',
       'INSURANCE_Medicare', 'INSURANCE_Private', 'INSURANCE_Self Pay',
       'GENDER_F', 'GENDER_M'],
      dtype='object')

## Comparison between sklearn and DAAL runtime for clinical features only

In [8]:
#applying clinical features alone
#applying smote
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())


In [10]:
#SVM without DAAL

with timeit():
    svm_model=SVC()
    svm_model.fit(X_train_res,y_train_res)
    print(classification_report(svm_model.predict(X_test),y_test))
    print(roc_auc_score(svm_model.predict(X_test),y_test))

              precision    recall  f1-score   support

          -1       0.97      0.88      0.93       339
           1       0.53      0.85      0.65        52

    accuracy                           0.88       391
   macro avg       0.75      0.87      0.79       391
weighted avg       0.91      0.88      0.89       391

0.8655547991831178
runtime: 0:00:00.211616


In [ ]:
#SVM for experiment (not to be considered as a final model)
train_model=d4p.svm_training(nClasses=2)
train_result=train_model.compute(X_train,y_train)
predict_algo = d4p.svm_prediction()
predict_result = predict_algo.compute(X_test, train_result.model)


In [11]:
#SVM with DAAL
with timeit():
    clf_d = daalsk.svm.SVC().fit(X_train_res, y_train_res)
    print(roc_auc_score(clf_d.predict(X_test),y_test))

0.8655547991831178
runtime: 0:00:00.172261


In [16]:
#logit (LASSO) without DAAL
with timeit():
    logit=LogisticRegression(penalty='l1').fit(X_train_res,y_train_res)
    print(roc_auc_score(logit.predict(X_test),y_test))

1.0
runtime: 0:00:00.026977


In [15]:
#logit (LASSO) with DAAL
with timeit():
    clf_logit=daalsk.linear_model.LogisticRegression(penalty='l1').fit(X_train_res,y_train_res)
    print(roc_auc_score(clf_logit.predict(X_test),y_test))

1.0
runtime: 0:00:00.026305


In [17]:
#Randomforest without DAAL
with timeit():
    rf=RandomForestClassifier(n_estimators=500).fit(X_train_res,y_train_res)
    print(roc_auc_score(rf.predict(X_test),y_test))

1.0
runtime: 0:00:01.643089


In [18]:
#Randomforest with DAAL
with timeit():
    clf_rf=daalsk.ensemble.RandomForestClassifier(n_estimators=500).fit(X_train_res,y_train_res)
    print(roc_auc_score(clf_rf.predict(X_test),y_test))

1.0
runtime: 0:00:00.213816


In [19]:
#KNN without DAAL
with timeit():
    knn=KNeighborsClassifier(n_neighbors=3).fit(X_train_res,y_train_res)
    print(roc_auc_score(knn.predict(X_test),y_test))
    

0.6131827731092437
runtime: 0:00:00.087359


In [20]:
#KNN with DAAL
with timeit():
    knn_daal=daalsknb.KNeighborsClassifier().fit(X_train_res,y_train_res)
    print(roc_auc_score(knn_daal.predict(X_test),y_test))


0.6195219795037756
runtime: 0:00:00.025731


## Comparison between sklearn and DAAL runtime for text based features only

In [5]:
#text features
from scipy import sparse
X_notes_tf = sparse.load_npz("text.npz")


In [11]:
#again splitting the notes into train/test
X_train, X_test, y_train, y_test = train_test_split(X_notes_tf, y_vitals, test_size=0.33, random_state=42,stratify=y_vitals)

In [12]:
#applying smote
sm = SMOTE(random_state=2)
X_train_res_text, y_train_res_text = sm.fit_sample(X_train, y_train.ravel())


In [24]:
#SVM without DAAL
with timeit():
    svm_model=SVC()
    svm_model.fit(X_train_res_text,y_train_res_text)
    print(classification_report(svm_model.predict(X_test),y_test))
    print(roc_auc_score(svm_model.predict(X_test),y_test))

              precision    recall  f1-score   support

          -1       0.98      0.92      0.95       327
           1       0.69      0.89      0.78        64

    accuracy                           0.92       391
   macro avg       0.83      0.91      0.86       391
weighted avg       0.93      0.92      0.92       391

0.9055571483180428
runtime: 0:00:04.432144


In [25]:
#SVM with DAAL
with timeit():
    clf_d = daalsk.svm.SVC().fit(X_train_res_text, y_train_res_text)
    print(classification_report(clf_d.predict(X_test),y_test))
    print(roc_auc_score(clf_d.predict(X_test),y_test))

              precision    recall  f1-score   support

          -1       0.98      0.92      0.95       327
           1       0.69      0.89      0.78        64

    accuracy                           0.92       391
   macro avg       0.83      0.91      0.86       391
weighted avg       0.93      0.92      0.92       391

0.9055571483180428
runtime: 0:00:04.248324


In [26]:
#logit (LASSO) without DAAL
with timeit():
    logit=LogisticRegression(penalty='l1').fit(X_train_res_text,y_train_res_text)
    print(roc_auc_score(logit.predict(X_test),y_test))

0.9258182100201644
runtime: 0:00:00.073211


In [27]:
#logit (LASSO) with DAAL
with timeit():
    clf_logit=daalsk.linear_model.LogisticRegression(penalty='l1').fit(X_train_res_text,y_train_res_text)
    print(roc_auc_score(clf_logit.predict(X_test),y_test))

0.9258182100201644
runtime: 0:00:00.071090


In [28]:
#Randomforest without DAAL
with timeit():
    rf=RandomForestClassifier(n_estimators=500).fit(X_train_res_text,y_train_res_text)
    print(roc_auc_score(rf.predict(X_test),y_test))

0.9506535947712417
runtime: 0:00:04.868175


In [29]:
#Randomforest with DAAL
with timeit():
    clf_rf=daalsk.ensemble.RandomForestClassifier(n_estimators=500).fit(X_train_res_text.todense(),y_train_res_text)
    print(roc_auc_score(clf_rf.predict(X_test),y_test))

0.9356209150326796
runtime: 0:00:00.383395


In [30]:
#KNN without DAAL
with timeit():
    knn=KNeighborsClassifier(n_neighbors=3).fit(X_train_res_text,y_train_res_text)
    print(roc_auc_score(knn.predict(X_test),y_test))

0.7419254658385093
runtime: 0:00:00.302171


In [31]:
#KNN with DAAL
with timeit():
    knn_daal=daalsknb.KNeighborsClassifier().fit(X_train_res_text.todense(),y_train_res_text)
    print(roc_auc_score(knn_daal.predict(X_test.todense()),y_test))

0.7605107616817218
runtime: 0:00:00.129535


## Comparison between sklearn and DAAL runtime for both clinical and text based features

In [7]:
df = pd.DataFrame(X_notes_tf.toarray())

In [8]:
#joint model
df_c = pd.concat([X_vitals.reset_index(drop=True), df], axis=1)


In [9]:
#again splitting the notes into train/test
X_train, X_test, y_train, y_test = train_test_split(df_c, y_vitals, test_size=0.33, random_state=42,stratify=y_vitals)

In [10]:
#applying smote
sm = SMOTE(random_state=2)
X_train_res_joint, y_train_res_joint = sm.fit_sample(X_train, y_train.ravel())


In [38]:
#SVM without DAAL
with timeit():
    svm_model=SVC()
    svm_model.fit(X_train_res_joint,y_train_res_joint)
    print(classification_report(svm_model.predict(X_test),y_test))
    print(roc_auc_score(svm_model.predict(X_test),y_test))

              precision    recall  f1-score   support

          -1       0.98      0.87      0.92       348
           1       0.46      0.88      0.60        43

    accuracy                           0.87       391
   macro avg       0.72      0.88      0.76       391
weighted avg       0.93      0.87      0.89       391

0.877205292702486
runtime: 0:00:01.494489


In [39]:
#SVM with DAAL
with timeit():
    clf_d = daalsk.svm.SVC().fit(X_train_res_joint, y_train_res_joint)
    print(classification_report(clf_d.predict(X_test),y_test))
    print(roc_auc_score(clf_d.predict(X_test),y_test))

              precision    recall  f1-score   support

          -1       0.98      0.87      0.92       348
           1       0.46      0.88      0.60        43

    accuracy                           0.87       391
   macro avg       0.72      0.88      0.76       391
weighted avg       0.93      0.87      0.89       391

0.877205292702486
runtime: 0:00:01.489457


In [40]:
#logit (LASSO) without DAAL
with timeit():
    logit=LogisticRegression(penalty='l1').fit(X_train_res_joint,y_train_res_joint)
    print(roc_auc_score(logit.predict(X_test),y_test))

1.0
runtime: 0:00:00.080190


In [41]:
#logit (LASSO) with DAAL
with timeit():
    clf_logit=daalsk.linear_model.LogisticRegression(penalty='l1').fit(X_train_res_joint,y_train_res_joint)
    print(roc_auc_score(clf_logit.predict(X_test),y_test))

1.0
runtime: 0:00:00.073157


In [42]:
#Randomforest without DAAL
with timeit():
    rf=RandomForestClassifier(n_estimators=500).fit(X_train_res_joint,y_train_res_joint)
    print(roc_auc_score(rf.predict(X_test),y_test))

0.9829505091167416
runtime: 0:00:03.170097


In [43]:
#Randomforest with DAAL
with timeit():
    clf_rf=daalsk.ensemble.RandomForestClassifier(n_estimators=500).fit(X_train_res_joint,y_train_res_joint)
    print(roc_auc_score(clf_rf.predict(X_test),y_test))

1.0
runtime: 0:00:00.275904


In [44]:
#KNN without DAAL
with timeit():
    knn=KNeighborsClassifier(n_neighbors=3).fit(X_train_res_joint,y_train_res_joint)
    print(roc_auc_score(knn.predict(X_test),y_test))

0.7201743901776497
runtime: 0:00:00.559542


In [15]:
#KNN with DAAL
with timeit():
    knn_daal=daalsknb.KNeighborsClassifier(n_neighbors=3).fit(X_train_res_text.todense(),y_train_res_text)
    print(roc_auc_score(knn_daal.predict(X_test.todense()),y_test))

0.7607118758434548
runtime: 0:00:00.132534
